In [1]:
import sys
import numpy as np
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel
from PyQt5.QtCore import QTimer
import tensorflow as tf

In [2]:
L = np.load("raw_05_11_15_53_11x.npy", allow_pickle=True)
mean_std = np.load("mean_std.npy", allow_pickle=True)
L = (L-mean_std[0])/mean_std[1]

model = tf.keras.models.load_model("LSTM_model.h5")

In [3]:
class App(QMainWindow):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.show()
    def initUI(self):
        self.setWindowTitle('Real-time prediction')
        self.setGeometry(100, 200, 1700, 400)
        # canvas thread
        obj = PlotCanvas(self, width=15, height=4)
        self.timer = QTimer(obj)
        self.timer.timeout.connect(obj.plotc)
        self.FRAME_PER_SECOND = 1
        self.timer.start(1000/self.FRAME_PER_SECOND)
        
class PlotCanvas(FigureCanvas):
    def __init__(self, parent=None, width=15, height=4, dpi=100):
        super().__init__( Figure(figsize=(width, height), dpi=dpi) )
        self.setParent(parent)
        self.data = [0 for i in range(200)]
        self.sec = [29 for i in range(200)]
        self.t = 0
    def plotc(self):
        global L, model
        new = np.array([ L[self.t*4+3-24+i] for i in range(25) ])
        ans = np.argmax(model.predict(new.reshape(1,25,1)))
        self.sec = self.sec[4:] + [ans for i in range(4)]
        self.data=self.data[4:] + [L[self.t*4]*8, L[self.t*4+1]*8, L[self.t*4+2]*8,\
            L[self.t*4+3]*8]
        fig = self.figure.add_subplot(111)
        fig.clear()
        fig.plot(self.data, 'r-')
        fig.plot(self.sec, 'g-')
        #fig.set_ylim(-2,2)
        fig.set_ylim(-5, 30)
        fig.set_title('Vibration')
        self.draw()
        self.t+=1

def data_collect():
    pass

In [4]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())

C:\Users\MH630\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:29: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


SystemExit: 0

C:\Users\MH630\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
